# 공원과 서울시 구/법정동 연결

### 1. 데이터 준비

In [122]:
# 1-1. 필요 라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [123]:
# 1-2. 데이터 파일 불러오기
# 법정동 경계파일 불러오기
dong_gdf = gpd.read_file\
    ('../4.Public_transportation/LSMD_ADM_SECT_UMD_Ldong/LSMD_ADM_SECT_UMD_11_202409.shp',
     encoding='cp949')

# 구 경계파일 불러오기
gu_gdf = gpd.read_file\
    ('../4.Public_transportation/LARD_ADM_SECT_SGG_gu/LARD_ADM_SECT_SGG_11_202405.shp',
     encoding='cp949')

# 구별 법정동 목록파일 불러오기
gu_dong_df = pd.read_csv\
    ('../4.Public_transportation/Gu_and_Legal_dong.csv', encoding='cp949')

# 공원 데이터 불러오기
park_df = pd.read_csv\
    ('../6.Infrastructure/공원좌표.csv', encoding='cp949')

### 2. 불러온 데이터 확인

In [124]:
print("=== 데이터 현황 ===")
print("\n[구 경계 데이터]")
print("- 데이터 크기:", gu_gdf.shape)
print("- 컬럼 목록:", gu_gdf.columns.tolist())
print("- 구 목록:", sorted(gu_gdf['SGG_NM'].tolist()))
print("- 좌표계:", gu_gdf.crs)

print("\n[법정동 경계 데이터]")
print("- 데이터 크기:", dong_gdf.shape)
print("- 컬럼 목록:", dong_gdf.columns.tolist())
print("- 좌표계:", dong_gdf.crs)

print("\n[공원 데이터]")
print("- 데이터 크기:", park_df.shape)
print("- 컬럼 목록:", park_df.columns.tolist())

print("\n[구별 법정동 데이터]")
print("- 데이터 크기:", gu_dong_df.shape)
print("- 컬럼 목록:", gu_dong_df.columns.tolist())

=== 데이터 현황 ===

[구 경계 데이터]
- 데이터 크기: (25, 5)
- 컬럼 목록: ['ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE', 'geometry']
- 구 목록: ['서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구']
- 좌표계: PROJCS["Korea_2000_Korea_Central_Belt_2010",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]



### 3. 데이터 전처리

In [125]:
# 3-1. 컬럼 정리
# 구_동 데이터 프레임 컬럼 정리
gu_dong_df = gu_dong_df[['gu', 'legal_dong']]
gu_dong_df.columns = ['gu', 'dong']
gu_dong_df['gu_dong'] = gu_dong_df['gu'] + '_' + gu_dong_df['dong']
gu_dong_df.info()
print(gu_dong_df['gu'].nunique())
print(gu_dong_df['dong'].nunique())
print(gu_dong_df['gu_dong'].nunique())

# 3-2. 공원 데이터 프레임 컬럼 정리
park_df.columns = ['park_name', 'X_GRS80TM', 'Y_GRS80TM', 'X_WGS84', 'Y_WGS84']
park_df.dropna()
park_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gu       467 non-null    object
 1   dong     467 non-null    object
 2   gu_dong  467 non-null    object
dtypes: object(3)
memory usage: 11.1+ KB
25
465
467
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   park_name  130 non-null    object 
 1   X_GRS80TM  119 non-null    float64
 2   Y_GRS80TM  119 non-null    float64
 3   X_WGS84    119 non-null    float64
 4   Y_WGS84    119 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


In [126]:
# 3-3. 좌표계 통일 (모두 EPSG:4326으로 변환)
dong_gdf = dong_gdf.to_crs("EPSG:4326")
gu_gdf = gu_gdf.to_crs("EPSG:4326")

# 3-4. 공원 데이터를 GeoDataFrame으로 변환
park_geometry = [Point(xy) for xy in zip(park_df['X_WGS84'], park_df['Y_WGS84'])]
park_gdf = gpd.GeoDataFrame(
    park_df, 
    geometry=park_geometry,
    crs="EPSG:4326"
)

print("\n=== 전처리 결과 ===")
print("- 법정동 좌표계:", dong_gdf.crs)
print("- 구 좌표계:", gu_gdf.crs)
print("- 공원 좌표계:", park_gdf.crs)


=== 전처리 결과 ===
- 법정동 좌표계: EPSG:4326
- 구 좌표계: EPSG:4326
- 공원 좌표계: EPSG:4326


### 4. 공간 분석 작업

In [127]:
# 4.1 공원과 구 매칭
parks_with_gu = gpd.sjoin(park_gdf, gu_gdf, how='left', predicate='intersects')

# 4-1 공원과 동 매칭
parks_with_dong = gpd.sjoin(park_gdf, dong_gdf, how='left', predicate='intersects')

# 4-3 최종 데이터프레임 생성
# 인덱스를 리셋하고 새로운 DataFrame 생성
result_df = pd.DataFrame({
    'park_name': parks_with_dong['park_name'].reset_index(drop=True),
    'gu': parks_with_gu['SGG_NM'],
    'dong': parks_with_dong['EMD_NM'].reset_index(drop=True)
})
result_df.info(), result_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   park_name  130 non-null    object
 1   gu         113 non-null    object
 2   dong       112 non-null    object
dtypes: object(3)
memory usage: 3.2+ KB


(None,
   park_name         gu   dong
 0      남산공원  서울특별시 용산구  용산동2가
 1    길동생태공원  서울특별시 강동구     길동
 2     서울대공원        NaN    NaN
 3       서울숲  서울특별시 성동구  성수동1가
 4     월드컵공원  서울특별시 마포구    상암동)

In [128]:
# 4-5. 동이 NaN인 행 지우기
result_df = result_df.dropna(subset=['dong'])
result_df = result_df.dropna(subset=['park_name'])

result_df.info(), result_df.head(), result_df.nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 118
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   park_name  112 non-null    object
 1   gu         112 non-null    object
 2   dong       112 non-null    object
dtypes: object(3)
memory usage: 3.5+ KB


(None,
    park_name         gu   dong
 0       남산공원  서울특별시 용산구  용산동2가
 1     길동생태공원  서울특별시 강동구     길동
 3        서울숲  서울특별시 성동구  성수동1가
 4      월드컵공원  서울특별시 마포구    상암동
 5  광화문시민열린마당  서울특별시 종로구    세종로,
 park_name    112
 gu            24
 dong          88
 dtype: int64)

In [129]:
# # 4-6. '서울특별시'와 '구' 이름 분리
# result_df['city'] = result_df['gu'].str.split(' ').str[0]
# result_df['gu'] = result_df['gu'].str.split(' ').str[1]
# result_df = result_df[['Id', 'park_stop_name', 'gu', 'dong']]
# result_df.nunique()

# 4-7. 특정 열의 모든 공백 제거
result_df['dong'] = result_df['dong'].str.replace(' ', '')
result_df.nunique()

park_name    112
gu            24
dong          88
dtype: int64

In [130]:
# 4-8. 구별 법정동 목록과 공원 데이터 매칭
result_df = pd.merge(gu_dong_df, result_df, how='outer', on=['dong'])
result_df.nunique(), result_df.info()
result_df = result_df[['park_name', 'gu_x', 'dong', 'gu_dong']]
result_df.columns = ['park_name', 'gu', 'dong', 'gu_dong']
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gu_x       493 non-null    object
 1   dong       493 non-null    object
 2   gu_dong    493 non-null    object
 3   park_name  116 non-null    object
 4   gu_y       116 non-null    object
dtypes: object(5)
memory usage: 19.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   park_name  116 non-null    object
 1   gu         493 non-null    object
 2   dong       493 non-null    object
 3   gu_dong    493 non-null    object
dtypes: object(4)
memory usage: 15.5+ KB


In [131]:
grouped_df = result_df.groupby('gu_dong')['park_name'].count().reset_index()
# # 출력 제한 해제
# pd.set_option('display.max_rows', None)  # 모든 행 출력
# 출력
print(grouped_df)

     gu_dong  park_name
0    강남구_개포동          0
1    강남구_논현동          0
2    강남구_대치동          0
3    강남구_도곡동          1
4    강남구_삼성동          1
..       ...        ...
462  중랑구_면목동          1
463   중랑구_묵동          0
464  중랑구_상봉동          0
465  중랑구_신내동          1
466  중랑구_중화동          0

[467 rows x 2 columns]


In [132]:
# 결과 확인
print("\n최종 데이터 정보:")
print(f"전체 행 수: {len(result_df)}개")
print(f"gu의 NaN 개수: {result_df['gu'].isna().sum()}개")
print(f"dong의 NaN 개수: {result_df['dong'].isna().sum()}개")
result_df.info(), result_df['gu'].nunique(), result_df['dong'].nunique(), \
    result_df['gu_dong'].nunique()


최종 데이터 정보:
전체 행 수: 493개
gu의 NaN 개수: 0개
dong의 NaN 개수: 0개
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   park_name  116 non-null    object
 1   gu         493 non-null    object
 2   dong       493 non-null    object
 3   gu_dong    493 non-null    object
dtypes: object(4)
memory usage: 15.5+ KB


(None, 25, 465, 467)

### 5. 서울시 모든 구별 법정동과 매칭

In [133]:
# 6-1. 공원이 있는 법정동
yes_park = result_df
print( yes_park['gu_dong'].nunique() )

# 6-2. 공원이 없는 법정동
# 빈 데이터프레임 생성 (final_df와 동일한 컬럼을 사용)
no_park = pd.DataFrame(columns=result_df.columns)
# park_name 열이 NaN인 행 필터링
na_rows = result_df[result_df['park_name'].isna()]
# 필터링된 행들을 not_park에 추가
no_park = pd.concat([no_park, na_rows], ignore_index=True)
no_park.info()
print(no_park['gu_dong'].nunique() )
print(no_park['gu_dong'].unique() )

467
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   park_name  0 non-null      object
 1   gu         377 non-null    object
 2   dong       377 non-null    object
 3   gu_dong    377 non-null    object
dtypes: object(4)
memory usage: 11.9+ KB
377
['구로구_가리봉동' '금천구_가산동' '종로구_가회동' '용산구_갈월동' '은평구_갈현동' '강동구_강일동' '강남구_개포동'
 '강서구_개화동' '송파구_거여동' '종로구_견지동' '종로구_경운동' '종로구_계동' '마포구_공덕동' '노원구_공릉동'
 '종로구_공평동' '강서구_공항동' '강서구_과해동' '종로구_관수동' '종로구_관철동' '종로구_관훈동' '중구_광희동1가'
 '중구_광희동2가' '종로구_교남동' '종로구_교북동' '종로구_구기동' '구로구_구로동' '은평구_구산동' '마포구_구수동'
 '광진구_구의동' '광진구_군자동' '구로구_궁동' '종로구_궁정동' '종로구_권농동' '성동구_금호동2가' '성북구_길음동'
 '종로구_낙원동' '서대문구_남가좌동' '중구_남대문로1가' '중구_남대문로2가' '중구_남대문로3가' '중구_남대문로4가'
 '중구_남대문로5가' '중구_남산동1가' '중구_남산동2가' '중구_남산동3가' '용산구_남영동' '중구_남창동' '중구_남학동'
 '관악구_남현동' '종로구_내수동' '종로구_내자동' '서대문구_냉천동' '마포구_노고산동' '은평구_녹번동' '강남구_논현동'
 '종로구_누상동' '종로구_누하동' '중구_다동' '영등포구_당산

In [134]:
# 1. 두 데이터프레임의 'gu_dong' 값들을 비교
final_combinations = set(gu_dong_df['gu_dong'].unique())
result_combinations = set(result_df['gu_dong'].unique())

# 2. result_df에는 있지만 final_df에는 없는 조합 확인
diff_combinations = result_combinations - final_combinations
print("result_df에만 있는 구-동 조합 수:", len(diff_combinations))
print("\n차이나는 조합들:")
print(sorted(list(diff_combinations)))

# 3. 원본 데이터 확인
print("\nfinal_df의 gu, dong 컬럼 구조:")
print(result_df[['gu', 'dong']].head())
print("\nresult_df의 gu, dong 컬럼 구조:")
print(result_df[['gu', 'dong']].head())

# 4. 두 데이터프레임의 'gu_dong' 생성 방식 확인
print("\nfinal_df의 gu_dong 생성 방식:")
if 'gu_dong' in result_df.columns:
    print(result_df[['gu', 'dong', 'gu_dong']].head())

result_df에만 있는 구-동 조합 수: 0

차이나는 조합들:
[]

final_df의 gu, dong 컬럼 구조:
    gu  dong
0  송파구   가락동
1  구로구  가리봉동
2  금천구   가산동
3  강서구   가양동
4  강서구   가양동

result_df의 gu, dong 컬럼 구조:
    gu  dong
0  송파구   가락동
1  구로구  가리봉동
2  금천구   가산동
3  강서구   가양동
4  강서구   가양동

final_df의 gu_dong 생성 방식:
    gu  dong   gu_dong
0  송파구   가락동   송파구_가락동
1  구로구  가리봉동  구로구_가리봉동
2  금천구   가산동   금천구_가산동
3  강서구   가양동   강서구_가양동
4  강서구   가양동   강서구_가양동


### 6. 결과 저장

In [135]:
# 공원이 있는 법정동
yes_park.to_csv('Data_preprocessing/seoul_park_locations_yes.csv', index=False, encoding='cp949')

# 공원이 없는 법정동
no_park.to_csv('Data_preprocessing/seoul_park_locations_no.csv', index=False, encoding='cp949')

# 둘을 종합한 법정동
result_df.to_csv('Data_preprocessing/seoul_park_locations_all.csv', index=False, encoding='cp949')

In [136]:
# 상세 위치 정보 출력
print("\n=== 구별 공원역 상세 목록 ===")
for gu in sorted(result_df['gu'].unique()):
    print(f"\n[{gu}]")
    gu_data = result_df[result_df['gu'] == gu].sort_values(['dong', 'park_name'])
    for _, row in gu_data.iterrows():
        print(f"- {row['park_name']}: {row['dong']}")


=== 구별 공원역 상세 목록 ===

[강남구]
- nan: 개포동
- nan: 논현동
- nan: 대치동
- 도곡근린공원: 도곡동
- 봉은공원: 삼성동
- nan: 세곡동
- nan: 수서동
- 도산근린공원: 신사동
- 신사근린공원: 신사동
- nan: 압구정동
- nan: 역삼동
- nan: 율현동
- 대모산도시자연공원: 일원동
- nan: 자곡동
- nan: 청담동

[강동구]
- nan: 강일동
- 동명근린공원: 고덕동
- 길동생태공원: 길동
- 일자산근린공원: 길동
- nan: 둔촌동
- 샛마을근린공원: 명일동
- 명일근린공원: 상일동
- nan: 성내동
- nan: 암사동
- 천호근린공원: 천호동

[강북구]
- nan: 미아동
- 북서울꿈의숲: 번동
- 오동근린공원: 번동
- nan: 수유동
- 솔밭근린공원: 우이동

[강서구]
- 궁산근린공원: 가양동
- 허준공원: 가양동
- nan: 개화동
- nan: 공항동
- nan: 과해동
- 우장산근린공원: 내발산동
- 매화근린공원: 등촌동
- nan: 마곡동
- 개화근린공원: 방화동
- 꿩고개근린공원: 방화동
- 방화근린공원: 방화동
- 염창근린공원: 염창동
- nan: 오곡동
- nan: 오쇠동
- nan: 외발산동
- 봉제산공원: 화곡동

[관악구]
- nan: 남현동
- 상도근린공원: 봉천동
- 샘말공원(관악산근린공원 샘말지구): 신림동

[광진구]
- 아차산공원: 광장동
- nan: 구의동
- nan: 군자동
- 어린이대공원: 능동
- nan: 자양동
- nan: 중곡동
- nan: 화양동

[구로구]
- nan: 가리봉동
- 개웅산근린공원: 개봉동
- 고척근린공원: 고척동
- nan: 구로동
- nan: 궁동
- nan: 신도림동
- 온수도시자연공원: 오류동
- nan: 온수동
- nan: 천왕동
- 푸른수목원: 항동

[금천구]
- nan: 가산동
- 감로천생태공원(관악산): 독산동
- 금천체육공원(관악산): 독산동
- 만수천공원(관악산): 독산동
- 금천폭포근린공원: 시흥동

[노원구]

### 7. 구별/동별 수 집계

In [137]:
# 7-1. 집계를 위한 Dataframe 만들기
all_park_dong = result_df.groupby(['gu', 'dong', 'gu_dong'])['park_name'].count().reset_index()
all_park_dong.columns = ['gu', 'dong', 'gu_dong', 'park_count']
all_park_dong

,gu,dong,gu_dong,park_count
0,강남구,개포동,강남구_개포동,0
1,강남구,논현동,강남구_논현동,0
2,강남구,대치동,강남구_대치동,0
3,강남구,도곡동,강남구_도곡동,1
4,강남구,삼성동,강남구_삼성동,1
...,...,...,...,...
462,중랑구,면목동,중랑구_면목동,1
463,중랑구,묵동,중랑구_묵동,0
464,중랑구,상봉동,중랑구_상봉동,0
465,중랑구,신내동,중랑구_신내동,1


In [138]:
# 7-2. 구별 공원 수 집계
# 7-2-1. 모든 구별 공원 수
park_gu_all = all_park_dong.groupby('gu')['park_count'].sum().reset_index()
# 7-2-2. 공원이 있는 구별 공원 수
park_gu_yes = park_gu_all[park_gu_all['park_count'] > 0]
# 7-2-3. 공원이 없는 구 목록
park_gu_no = park_gu_all[park_gu_all['park_count'] == 0]

park_gu_all.info(), park_gu_yes.info(), park_gu_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          25 non-null     object
 1   park_count  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 24
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          24 non-null     object
 1   park_count  24 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 576.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 8 to 8
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          1 non-null      object
 1   park_count  1 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 24.0+ bytes


(None, None, None)

In [139]:
# 7-3. 동별 공원 수 집계
# 7-3-1. 모든 동별 공원 수
park_dong_all = all_park_dong.groupby(['gu', 'dong', 'gu_dong'])['park_count'].sum().reset_index()
# 7-3-2. 공원이 있는 동별 공원 수
park_dong_yes = park_dong_all[park_dong_all['park_count'] > 0]
# 7-3-3. 공원이 없는 동 목록
park_dong_no = park_dong_all[park_dong_all['park_count'] == 0]

park_dong_all.info(), park_dong_yes.info(), park_dong_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          467 non-null    object
 1   dong        467 non-null    object
 2   gu_dong     467 non-null    object
 3   park_count  467 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 14.7+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 3 to 465
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          90 non-null     object
 1   dong        90 non-null     object
 2   gu_dong     90 non-null     object
 3   park_count  90 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 3.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 377 entries, 0 to 466
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gu          377 non-null    objec

(None, None, None)

### 8. 집계 파일 저장

In [140]:
park_gu_all.to_csv('Data_Preprocessing/6_park_gu_all.csv', index = False)
park_gu_yes.to_csv('Data_Preprocessing/6_park_gu_yes.csv', index = False)
park_gu_no.to_csv('Data_Preprocessing/6_park_gu_no.csv', index = False)
park_dong_all.to_csv('Data_Preprocessing/6_park_dong_all.csv', index = False)
park_dong_yes.to_csv('Data_Preprocessing/6_park_dong_yes.csv', index = False)
park_dong_no.to_csv('Data_Preprocessing/6_park_dong_no.csv', index = False)